Bag: Parallel Lists for semi-structured data
========================

NumPy and Pandas excel on structured and homogenous data (e.g. an array of floats).
NumPy and Pandas are not designed for data that is highly nested, variable length, heterogeneous, or has missing values. (We'll call this kind of data "messy".)

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or `toolz` let us process large volumes in a small space.  If we combine this with multiprocessing then we can churn through a fair amount of data.

Dask.bag is a high level dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + multiprocessing
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag.html#api)

Creation
----------

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.. 

In [ ]:
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
import os
b = db.from_filenames(os.path.join('data', 'accounts.*.json.gz'))

In [ ]:
# Requires `boto` library
b = db.from_s3('s3://nyqpug/tips.csv')

Manipulation
--------------

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

As with `Array` and `DataFrame` objects, operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution.  

Dask.bag uses `dask.multiprocessing.get` by default.  For examples using dask.bag with a distributed cluster see 

*  [Dask.distributed docs](http://dask.readthedocs.org/en/latest/distributed.html)
*  [Blogpost analyzing github data on EC2](http://continuum.io/blog/dask-distributed-cluster)

In [ ]:
def iseven(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(iseven).map(lambda x: x ** 2)
c

In [ ]:
c.compute()

Exercise: Accounts JSON data
---------------------------------

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [ ]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.from_filenames(filename)
lines.take(3)

Our data comes out of the file as lines of text.  We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [ ]:
import json
js = lines.map(json.loads)
js.take(3)

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [ ]:
js.filter(lambda record: record['name'] == 'Alice').take(5)

In [ ]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

In [ ]:
# Average number of transactions for all of the Alice's
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

### Use `concat` to de-nest

In the example below we see the use of `concat` to flatten results.  We compute the average amount for all transactions for all Alices.

In [ ]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .concat()
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .concat()
   .pluck('amount')
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .concat()
   .pluck('amount')
   .mean()
   .compute())

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full on-disk shuffling of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad.  In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

In [ ]:
b.groupby(lambda x: x % 2).map(lambda k, v: (k, max(v))).compute()

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.org/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [ ]:
b.foldby(lambda x: x % 2, binop=lambda acc, x: acc if acc > x else x,
                          combine=lambda acc1, acc2: acc1 if acc1 > acc2 else acc2).compute()

### Example with account data

We find the number of people with the same name.

In [ ]:
%%time

# Warning, this one takes a while...
js.groupby('name').map(lambda (k, v): (k, len(v))).compute()

In [ ]:
%%time

# This one is comparatively fast and produces the same result.
js.foldby(key='name', binop=lambda total, x: total + 1, 
                      initial=0, 
                      combine=lambda a, b: a + b, 
                      combine_initial=0).compute()

### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

DataFrames
------------

Pandas is faster than Pure Python.  In the same way `dask.dataframe` can be faster than `dask.bag`.  You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [ ]:
df = js.to_dataframe()
df.head()

In [ ]:
df.groupby('name').transactions.count().compute()

This example is less-than-optimal for two reasons

1.  The `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..  

2.  Dask.dataframe uses `threading` by default, not `multiprocessing`, so we end up losing the benefits of multiprocessing when executing the `dask.bag` portion of our graph.